# Content Design for RAG
This notebook is part of a collection of material related to content design principles for retrieval-augmented generation (RAG).

You can explore the complete collection here: [Content Design for RAG on GitHub](https://github.com/spackows/ICAAI-2024_RAG-CD/blob/main/README.md)

**Example scenario**

Imagine your company sells seeds and gardening supplies online.  On your website, you have articles with gardening information and advice.  You are building a RAG solution for your company website that can answer customer questions about your products, using your website articles as a knowledge base.

# Prompt by question type
To achieve best results, you can design your RAG solution to handle each type of submitted question differently.

This sample notebook demonstrates a simple approach this problem: using a distinct LLM prompt for each question type.

**Contents**
1. Write prompt text
2. Prompt an LLM
3. Test input

## 1. Write prompt text

The following prompt templates handle two types of questions:
- "what-is" questions asking for factual details or for an explanation of a concept or idea
- "how-to" questions asking for instructions on how to do something

In [17]:
g_whatis_template = """Article: 
----
%s
----

Answer the following question using only information from the article. 
If there is no good answer in the article, say: "No answer was found" 

Question: %s
Answer: """

In [56]:
g_howto_template = """Article: 
----
%s
----

Answer the following question as a list of numbered steps using only information from the article. 
If there is no good answer in the article, say: "No answer was found"

Question: What is the best way to water cucumbers?
Answer: To water cucumbers, perform these steps: 
1. Apply water to the base of the plant, to avoid fungal diseases on damp foliage
2. Water frequently, preferably with a drip or soaker hose
3. Avoid allowing soil to completely dry out
4. Use mulch to keep soil moist

Question: How do you prevent slugs eating holes in peppers
Answer: To prevent slugs eating holes in peppers, perform these steps: 
1. Surround your plant with a copper barrier (eg. tape)
2. Place jagged material such as crushed eggshells or seashells around your plant
3. Manually remove slugs in the evening or early morning

Question: how to transpant a seedling tomato?
Answer: To transpant a seedling tomato, perform these steps: 
1. Harden off the seedlings to avoid transplant shock
2. Prune flowers to send energy to growing roots
3. Pre-moisten the soil
4. Plant seedlings on an angle or deep enough to burying the lowest few stems
5. Water thoroughly

Question: %s 
Answer: """

## 2. Prompt an LLM

See: [Foundation models Python library](https://ibm.github.io/watson-machine-learning-sdk/foundation_models.html)

### Prerequisites
Before you can prompt a foundation model in watsonx.ai, you must perform the following setup tasks:
- 2.1 Create an instance of the Watson Machine Learning service
- 2.2 Associate the Watson Machine Learning instance with the current project
- 2.3 Create an IBM Cloud API key
- 2.4 Look up the current project ID


#### 2.1 Create an instance of the Watson Machine Learning service
If you don't already have an instance of the IBM Watson Machine Learning service, you can create an instance of the service from the IBM Cloud catalog: [Watson Machine Learning service](https://cloud.ibm.com/catalog/services/watson-machine-learning)

#### 2.2 Associate an instance of the Watson Machine Learning service with the current project
The current project is the project in which you are running this notebook.

If an instance of Watson Machine Learning is not already associated with the current project, follow the instructions in this topic to do so: [Adding associated services to a project](https://dataplatform.cloud.ibm.com/docs/content/wsj/getting-started/assoc-services.html?context=wx&audience=wdp)

#### 2.3 Create an IBM Cloud API key
Create an IBM Cloud API key by following these instruction: [Creating an IBM Cloud API key](https://cloud.ibm.com/docs/account?topic=account-userapikey&interface=ui#create_user_key)

Then paste your new IBM Cloud API key in the code cell below.

In [7]:
cloud_apikey = ""

g_wml_credentials = { 
    "url"    : "https://us-south.ml.cloud.ibm.com", 
    "apikey" : cloud_apikey
}

#### 2.4 Look up the current project ID
The current project is the project in which you are running this notebook. You can get the ID of the current project programmatically by running the following cell.

In [8]:
import os

g_project_id = os.environ["PROJECT_ID"]

Just FYI: List supported models

In [9]:
from ibm_watson_machine_learning.foundation_models.utils.enums import ModelTypes

model_ids = list( map( lambda e: e.value, ModelTypes._member_map_.values() ) )
model_ids

['google/flan-t5-xxl',
 'google/flan-ul2',
 'bigscience/mt0-xxl',
 'eleutherai/gpt-neox-20b',
 'ibm/mpt-7b-instruct2',
 'bigcode/starcoder',
 'meta-llama/llama-2-70b-chat',
 'meta-llama/llama-2-13b-chat',
 'ibm/granite-13b-instruct-v1',
 'ibm/granite-13b-chat-v1',
 'google/flan-t5-xl',
 'ibm/granite-13b-chat-v2',
 'ibm/granite-13b-instruct-v2',
 'elyza/elyza-japanese-llama-2-7b-instruct',
 'ibm-mistralai/mixtral-8x7b-instruct-v01-q',
 'codellama/codellama-34b-instruct-hf',
 'ibm/granite-20b-multilingual']

Now prompt an LLM ...

In [34]:
from ibm_watson_machine_learning.foundation_models import Model
import json

def answer( model_id, prompt_parameters, prompt_template, article_txt, question_txt, b_debug=False ):
    model = Model( model_id, g_wml_credentials, prompt_parameters, g_project_id )
    prompt_text = prompt_template % ( article_txt, question_txt )
    raw_response = model.generate( prompt_text )
    if b_debug:
        print( "prompt_text:\n'" + prompt_text + "'\n" )
        print( "raw_response:\n" + json.dumps( raw_response, indent=3 ) )
    if ( "results" in raw_response ) \
       and ( len( raw_response["results"] ) > 0 ) \
       and ( "generated_text" in raw_response["results"][0] ):
        return raw_response["results"][0]["generated_text"]
    else:
        return ""

In [68]:
model_id = "google/flan-t5-xxl"

prompt_parameters = {
    "decoding_method" : "sample",
    "min_new_tokens"  : 0,
    "max_new_tokens"  : 300,
    "random_seed"     : 2391547291
}

article_txt = """
## Growing peppers in containers
When it comes to growing green peppers in containers, the more room the plants have, the better.
Pepper plants need 18 - 24 inches of width, and their roots need 14 to 24 inches of depth.
The type of container doesn't matter: clay or plastic pots, wooden boxes, plastic totes, fabric grow bags, or even garbage bins.
"""
question_txt = "how large a pot do I need for growing peppers"

answer_txt = answer( model_id, prompt_parameters, g_whatis_template, article_txt, question_txt )
print( "\"What-is\" question:\n" + question_txt + "\n\nAnswer:\n" + answer_txt.strip() + "\n" )

"What-is" question:
how large a pot do I need for growing peppers

Answer:
Pepper plants need 18 - 24 inches of width, and their roots need 14 to 24 inches of depth



In [69]:
model_id = "meta-llama/llama-2-70b-chat"

prompt_parameters = {
    "decoding_method" : "sample",
    "min_new_tokens"  : 0,
    "max_new_tokens"  : 300,
    "stop_sequences"  : [ "\n\n" ],
    "random_seed"     : 3500937994
}

article_txt = """
There are multiple ways to harvest spinach.
Once there are multiple leaves and the plant is more than 4 inches tall, you can begin harvesting.
One option is to cut mature leaves on the outside of the plant with scissors.  This way, you can harvest continuously.
To harvest in bulk, gather the plant in one hand and cut the plant with a knife above the crown.  More leaves will be ready to harvest in a few weeks.
Later in the season, when the temperature is warmer, the plant will bolt or go to seed.  The taste of the leaves changes then, so you should harvest the whole plant as soon as you see signs it is bolting.
"""
question_txt = "How can I harvest spinach?"
answer_txt = answer( model_id, prompt_parameters, g_howto_template, article_txt, question_txt )
print( "\"How-to\" question:\n" + question_txt + "\n\nAnswer:\n" + answer_txt.strip() + "\n" )

"How-to" question:
How can I harvest spinach?

Answer:
1. Cut mature leaves on the outside of the plant with scissors. 
2. Gather the plant in one hand and cut the plant with a knife above the crown. 
3. If the plant has bolted, harvest the whole plant.

